In [1]:
import random

P = 41

def encode(x):
    return x % P

def decode(x):
    return x if x <= P/2 else x - P

In [2]:
x = encode(-5)
print("encoded: %d" % x)
print("decoded: %d" % decode(x))

encoded: 36
decoded: -5


In [14]:
# Extended Euclidian Alg
# ax + by= gcd(a,b)
# return gcd(a,b), a, b

def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def egcd_binary(a,b):
    u, v, s, t, r = 1, 0, 0, 1, 0
    while (a % 2 == 0) and (b % 2 == 0):
        a, b, r = a//2, b//2, r+1
    alpha, beta = a, b
    while (a % 2 == 0):
        a = a//2
        if (u % 2 == 0) and (v % 2 == 0):
            u, v = u//2, v//2
        else:
            u, v = (u + beta)//2, (v - alpha)//2
    while a != b:
        if (b % 2 == 0):
            b = b//2
            if (s % 2 == 0) and (t % 2 == 0):
                s, t = s//2, t//2
            else:
                s, t = (s + beta)//2, (t - alpha)//2
        elif b < a:
            a, b, u, v, s, t = b, a, s, t, u, v
        else:
            b, s, t = b - a, s - u, t - v
    return (2 ** r) * a, s, t


def inverse(a):
    _, b, _ = egcd_binary(a, P)
    return b

## Polynomials
#### Coeffecient representation and Lagrange polynomial
Two types of polynomial representations:
- f(x) = a0 + a1x + a2x^2 + ... + anx^n
- [1, f(1)], [2, f(2)], ... [n+1, f(n+1)]


In [15]:
# using Horner's rule
# f(x) = a0 + a1x + a2x^2 + ... + anx^n
#      = a0 + x (a1 + x (a2 + x (....(an + x))))

def evaluate_at_point(coefs, point):
    result = 0
    for coef in reversed(coefs):
        result = (coef + point * result) % P
    return result

# see https://en.wikipedia.org/wiki/Lagrange_polynomial
# lj(x) = [(x - x1)/(xj - x1)][(x - x2)/(xj - x2)]....[(x - xn)/(xj - xn)]
# return l0(point), l1(point), ... ln(pont)

def lagrange_constants_for_point(points, point):
    constants = [0] * len(points)
    for i in range(len(points)):
        xi = points[i]
        num = 1
        denum = 1
        for j in range(len(points)):
            if j != i:
                xj = points[j]
                num = (num * (xj - point)) % P
                denum = (denum * (xj - xi)) % P
        constants[i] = (num * inverse(denum)) % P
    return constants

# return f(point) given (points, values) pairs that represent f
def interpolate_at_point(points_values, point):
    points, values = zip(*points_values)
    constants = lagrange_constants_for_point(points, point)
    return sum( vi * ci for vi, ci in zip(values, constants) ) % P

## Shamir sharing

In [16]:
# shre among 10 parites
# need more than 4 parties to recover

N = 10
SHARE_POINTS = [ p for p in range(1, N+1) ]

T = 4
def sample_shamir_polynomial(zero_value):
    coefs = [zero_value] + [random.randrange(P) for _ in range(T)]
    return coefs

In [17]:
# generate poly and plug in points to caculate shares

def shamir_share(secret):
    polynomial = sample_shamir_polynomial(secret)
    shares = [ evaluate_at_point(polynomial, p) for p in SHARE_POINTS ]
    return shares

# reconstruct by lagrange interpolation

def shamir_reconstruct(shares):
    polynomial = [ (p,v) for p,v in zip(SHARE_POINTS, shares) if v is not None ]
    secret = interpolate_at_point(polynomial, 0)
    return secret

#### Sharing works!

In [18]:
shares = shamir_share(5)

for i in range(N-(T+1)):
    shares[i] = None
    
#shares[-1] = None  # would fail; we need T+K points to reconstruct
x = shamir_reconstruct(shares)
assert(x == 5)

#### Operations  and Class

In [19]:
# x, y are shares array
# all return resultant z array

def shamir_add(x, y):
    return [ (xi + yi) % P for xi, yi in zip(x, y) ]

def shamir_sub(x, y):
    return [ (xi - yi) % P for xi, yi in zip(x, y) ]

def shamir_mul(x, y):
    return [ (xi * yi) % P for xi, yi in zip(x, y) ]

In [22]:
class Shamir:
    
    def __init__(self, secret=None):
        # generate shares as initialized
        # record initial degree
        self.shares = shamir_share(encode(secret)) if secret is not None else []
        self.degree = T
    
    def reveal(self):
        # total shares have to be less than N in order for N parties to reconstruct
        assert(self.degree+1 <= N)
        # use all shares to reconstruct regardless
        return decode(shamir_reconstruct(self.shares))
    
    def __repr__(self):
        print("shares", self.shares)
        return "Shamir(%d)" % self.reveal()
    
    # construct Shamir class to as the result
    # set class var via add, sub and mul functions
    
    def __add__(x, y):
        z = Shamir()
        z.shares = shamir_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Shamir()
        z.shares = shamir_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Shamir()
        z.shares = shamir_mul(x.shares, y.shares)
        # don't need to bring down degree here as long as total degree < N
        z.degree = x.degree + y.degree
        return z

In [26]:
x = Shamir(2)
print(x)

y = Shamir(3)
print(y)

z = x - y
print("\nSub ", z)
assert(z.reveal() == -1)

v = x * y
print("\nMul", v)
assert(v.reveal() == 6)

shares [11, 29, 31, 27, 21, 11, 30, 23, 11, 9]
Shamir(2)
shares [31, 25, 18, 12, 19, 20, 6, 19, 29, 16]
Shamir(3)

Sub  shares [21, 4, 13, 15, 2, 32, 24, 4, 23, 34]
Shamir(-1)

Mul shares [13, 28, 25, 37, 30, 15, 16, 27, 32, 21]
Shamir(6)


In [28]:
# Can't mul again since degree exceed
# w.degree = x.degree + v.degree = 4 + 8 = 12 > 10
w = x * v
print(w)

shares [20, 33, 37, 15, 15, 1, 29, 6, 24, 25]


AssertionError: 

## Packed sharing

In [29]:
# shares among 20 parties
N = 20
# T and K together defines the poly, 10 pairs in total
# 8 points of random values
T = 8
# 2 points that actually evaluates to the securet
K = 2

assert(T+K <= N)

In [30]:
SECRET_POINTS =     [ -p % P for p in range(1, K+1) ]
RANDOMNESS_POINTS = [ -p % P for p in range(K+1, K+T+1) ]
assert(set(SECRET_POINTS).intersection(RANDOMNESS_POINTS) == set())

# return 10 pairs that uniquly define the poly
def sample_packed_polynomial(secrets):
    assert(len(secrets) == K)
    # [1, 2] + [3, 4, ... 10]
    points = SECRET_POINTS + RANDOMNESS_POINTS
    # [secret1, secret2] + [random3, random4,...., random10]
    values = secrets + [ random.randrange(P) for _ in range(T) ]
    # (1, secret1), (2, secret2), (3, random3)....
    return list(zip(points, values))

In [31]:
SHARE_POINTS = [ p for p in range(1, N+1) ]
assert(set(SHARE_POINTS).intersection(SECRET_POINTS) == set())
assert(set(SHARE_POINTS).intersection(RANDOMNESS_POINTS) == set())

def packed_share(secrets):
    # define lagrange poly
    polynomial = sample_packed_polynomial(secrets)
    # evalute f(x1), f(x2),...,f(x20) as your shares
    shares = [ interpolate_at_point(polynomial, p) for p in SHARE_POINTS ]
    return shares

def packed_reconstruct(shares):
    points = SHARE_POINTS
    values = shares
    #(x1, share1), (x2, share2), ...,(x20, share20) forms the same lagrange poly
    # because it's unique to all sampled points
    polynomial = [ (p,v) for p,v in zip(points, values) if v is not None ]
    # just do the opposite
    # passing in points = [points where secrets are located] to get secret
    secrets = [ interpolate_at_point(polynomial, p) for p in SECRET_POINTS ]
    return secrets


#### Packed works!

In [32]:
secrets = [5,6]
shares = packed_share(secrets)
for i in range(N-(T+K)):
    shares[i] = None
#shares[-1] = None  # would fail; we need T+K points to reconstruct
reconstructed_secrets = packed_reconstruct(shares)
assert(reconstructed_secrets == secrets)

#### Operations and Class

In [37]:
def packed_add(x, y):
    return [ (xi + yi) % P for xi, yi in zip(x, y) ]

def packed_sub(x, y):
    return [ (xi - yi) % P for xi, yi in zip(x, y) ]

def packed_mul(x, y):
    return [ (xi * yi) % P for xi, yi in zip(x, y) ]

In [38]:
class Packed:
    
    def __init__(self, secrets=None):
        self.shares = packed_share([ encode(s) for s in secrets ]) if secrets is not None else []
        self.degree = T+K-1
    
    def reveal(self):
        assert(self.degree+1 <= N)
        #print(packed_reconstruct(self.shares))
        return [ decode(s) for s in packed_reconstruct(self.shares) ]
    
    def __repr__(self):
        print(self.shares)
        return "Packed(%s)" % self.reveal()
    
    def __add__(x, y):
        z = Packed()
        z.shares = packed_add(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __sub__(x, y):
        z = Packed()
        z.shares = packed_sub(x.shares, y.shares)
        z.degree = max(x.degree, y.degree)
        return z
    
    def __mul__(x, y):
        z = Packed()
        z.shares = packed_mul(x.shares, y.shares)
        z.degree = x.degree + y.degree
        return z

In [39]:
x = Packed([2,3])
print(x)

y = Packed([2,3])
print(y)

z = x - y
print(z)
assert(z.reveal() == [0,0])

v = x * y
print(v)
assert(v.reveal() == [4,9])

[36, 2, 35, 29, 32, 18, 37, 5, 40, 33, 3, 21, 22, 39, 2, 4, 39, 3, 27, 31]
Packed([2, 3])
[12, 33, 26, 14, 15, 14, 2, 13, 27, 38, 35, 38, 30, 24, 2, 29, 19, 20, 38, 19]
Packed([2, 3])
[24, 10, 9, 15, 17, 4, 35, 33, 13, 36, 9, 24, 33, 15, 0, 16, 20, 24, 30, 12]
Packed([0, 0])
[22, 25, 8, 37, 29, 6, 33, 24, 14, 24, 23, 19, 4, 34, 4, 34, 3, 19, 1, 15]
Packed([4, 9])
